In [1]:
import sys
import os

_path_added = False

def add_project_root_to_path():
    """
    Check if the project root directory is in the Python path.
    If not, add it to sys.path and change the working directory to the project root.
    """
    global _path_added

    if _path_added:
        return
    
    root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

    if root_dir not in sys.path:
        sys.path.append(root_dir)
        print(f"Added {root_dir} to Python path.")
    
    if os.getcwd() != root_dir:
        os.chdir(root_dir)
        print(f"Changed working directory to {root_dir}.")
    _path_added = True


add_project_root_to_path()

Added /home/developing_nacho/fhdw/web_social_analytics/dj-gpt/insta_scraping to Python path.
Changed working directory to /home/developing_nacho/fhdw/web_social_analytics/dj-gpt/insta_scraping.


In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dorfterror_database.database_structure import Base, Post, Profile
from pathlib import Path

root_path = Path(".").resolve()
print(root_path)

engine = create_engine(f"sqlite:///{root_path}/dorfterror_database/dorfterror.db")
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)
session = Session()

/home/developing_nacho/fhdw/web_social_analytics/dj-gpt/insta_scraping


In [22]:
total_posts = session.query(Post).count()

# Top 25% erfolgreichste Posts
top_posts = (
    session.query(Post)
    .order_by(Post.like_count.desc())
    .limit( int(total_posts * 0.25) )
    .all()
)

print(top_posts)

[<dorfterror_database.database_structure.Post object at 0x7a5650336de0>, <dorfterror_database.database_structure.Post object at 0x7a5650337410>, <dorfterror_database.database_structure.Post object at 0x7a5650336960>, <dorfterror_database.database_structure.Post object at 0x7a56503371d0>, <dorfterror_database.database_structure.Post object at 0x7a5650334200>, <dorfterror_database.database_structure.Post object at 0x7a56503350d0>, <dorfterror_database.database_structure.Post object at 0x7a56503341a0>, <dorfterror_database.database_structure.Post object at 0x7a5650335850>, <dorfterror_database.database_structure.Post object at 0x7a56503357f0>, <dorfterror_database.database_structure.Post object at 0x7a5650335160>, <dorfterror_database.database_structure.Post object at 0x7a56503370b0>, <dorfterror_database.database_structure.Post object at 0x7a5650337b60>, <dorfterror_database.database_structure.Post object at 0x7a5650334440>, <dorfterror_database.database_structure.Post object at 0x7a5650

In [23]:
from openai import OpenAI

# Initialize LM Studio client
client = OpenAI(base_url="http://192.168.178.85:1200/v1", api_key="lm-studio")
MODEL = "google/gemma-3-4b"


In [24]:
def prepare_weighted_captions(posts):
    """
    Bereitet die Captions mit Gewichtung basierend auf Like-Anteil vor.
    """
    # Gesamtsumme der Likes berechnen
    total_likes = sum(post.like_count for post in posts if post.like_count)
    if total_likes == 0:
        raise ValueError("Keine Likes gefunden – überprüfe die Daten.")

    # Berechne Gewicht für jede Caption
    weighted_captions = []
    for post in posts:
        if post.caption and post.like_count:
            weight = post.like_count / total_likes
            # Multipliziere die Caption proportional (Skalierungsfaktor 10 für mehr Wirkung)
            repeats = int(weight * 10 * len(posts))  # z.B. bei 5% Gewicht ≈ 2-3 Wiederholungen
            weighted_captions.extend([post.caption] * max(1, repeats))
    
    return weighted_captions

def chunk_list(data, chunk_size):
    """Teile die Liste in kleinere Chunks auf."""
    for i in range(0, len(data), chunk_size):
        yield data[i:i + chunk_size]


In [29]:

def analyze_style_and_content(weighted_captions, chunk_size=100, model=MODEL):
    """
    Lässt das Modell Schreibstil, Tonfall, Themen und Hashtags analysieren.
    """
    combined_analysis = []

    for chunk in chunk_list(weighted_captions, chunk_size):
        # Alle gewichteten Captions zusammenführen
        joined_captions = "\n".join([f"- {caption}" for caption in chunk])

        # Prompt zur Analyse
        prompt = (
            "Du bist ein Social Media Experte für eine Band. Analysiere die folgenden Instagram-Captions "
            "und beschreibe ausführlich:\n"
            "1. Den Schreibstil (z.B. locker, emotional, witzig)\n"
            "2. Den typischen Tonfall (z.B. motivierend, humorvoll)\n"
            "3. Häufig verwendete Wörter oder Phrasen\n"
            "4. Beliebte Themen in den Posts\n"
            "5. Häufig genutzte Emojis und Hashtags\n\n"
            "Hier sind die Captions (mehrfach wiederholt, je nach Wichtigkeit):\n"
            "---\n"
            f"{joined_captions}\n"
            "---\n\n"
            "Gib deine Analyse als strukturierte Liste aus."
        )

        # Anfrage an LM Studio
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=800,
            timeout=60
        )

        analysis = response.choices[0].message.content.strip()
        print(f"📦 Analyse für Chunk {idx}:\n{analysis}\n{'-'*50}\n")
        combined_analysis.append(analysis)
    
    return "\n".join(combined_analysis)


def generate_new_caption(style_analysis, new_context, model=MODEL):
    """
    Nutzt die Stil-Analyse und den neuen Kontext, um neue Captions zu generieren.
    """
    prompt = (
        "Basierend auf der folgenden Analyse des Schreibstils und der Inhalte unserer bisherigen Instagram-Posts:\n\n"
        f"{style_analysis}\n\n"
        "Wir möchten einen neuen Post veröffentlichen mit folgendem Kontext:\n"
        f"{new_context}\n\n"
        "💡 Erstelle 3 kreative Instagram-Captions, die den beschriebenen Stil und Tonfall übernehmen, "
        "die beliebten Themen und Hashtags einbeziehen und maximal 2 Sätze lang sind. "
        "Verwende Emojis passend zum Inhalt und füge bis zu 3 Hashtags hinzu. Gib die Captions als nummerierte Liste aus."
    )

    # Anfrage an LM Studio
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.95,
        max_tokens=500
    )

    return response.choices[0].message.content.strip()


In [30]:
# --- 🔥 Anwendung ---
# 1. Alle Posts aus DB
#all_posts = session.query(Post).filter(Post.caption.isnot(None)).all()
all_posts = top_posts

# 2. Gewichtete Captions vorbereiten
weighted_captions = prepare_weighted_captions(all_posts)

# 3. Schreibstil und Inhalte analysieren
style_analysis = analyze_style_and_content(weighted_captions)

print("📊 Analyse des Schreibstils und der Inhalte:\n")
print(style_analysis)

# 4. Neue Caption generieren
new_context = "Wir haben gerade ein Konzert auf dem Dalmstöckle Open Air gespielt und möchten dazu Bilder posten."
new_captions = generate_new_caption(style_analysis, new_context)

print("\n✨ Vorschläge für neue Captions:\n")
print(new_captions)

APITimeoutError: Request timed out.

In [ ]:
# Funktion: Extrahiere Themen aus Captions
def get_themes_from_captions(captions_block):
    joined_captions = "\n".join([f"- {c}" for c in captions_block])

    prompt = (
        "Die folgenden Instagram-Captions stammen von sehr erfolgreichen Posts. "
        "Analysiere diese Captions und liste die Hauptthemen auf, die sie gemeinsam haben. "
        "Fasse ähnliche Themen zusammen und gib sie als nummerierte Liste aus.\n\n"
        f"{joined_captions}"
    )

    # Anfrage an LM Studio senden
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "Du bist ein Social-Media-Analyst."},
            {"role": "user", "content": prompt},
        ],
        temperature=0.5,
        max_tokens=500
    )

    # Antwort zurückgeben
    return response.choices[0].message.content

In [9]:
def chunk_list(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

caption_chunks = list(chunk_list(captions, 5))  # z.B. 5 Captions pro Chunk

all_themes = []
for i, block in enumerate(caption_chunks):
    print(f"\n📝 Block {i + 1} Themen:")
    themes = get_themes_from_captions(block)
    print(themes)
    all_themes.append(themes)

# Optional: Alle Themen aus allen Chunks zusammenfassen
print("\n\n=== Zusammenfassung aller Themen aus allen Chunks ===")
print("\n".join(all_themes))


📝 Block 1 Themen:
Okay, hier ist eine Analyse der Instagram-Captions von Dorfterror, die auf die Hauptthemen fokussiert ist und diese zusammenfasst:

**Hauptthemen & Zusammenfassungen:**

1.  **Musikalische Aktivität & Releases:** Dies ist das dominierendste Thema. Die Band stellt regelmäßig neue Musik (EPs, Musikvideos) vor und teilt Streaming-Links. Es geht um die aktive Verbreitung ihrer Musik.
2.  **Live-Musik & Tournee:** Dorfterror legt großen Wert auf Live-Auftritte. Sie teilen regelmäßig Konzerttermine, Ankündigungen für Festivals und eigene Shows. Die Band betont das gemeinsame Musizieren und die Bühnenpräsenz.
3.  **Community & Fan Engagement:** Ein zentraler Punkt ist der Aufbau einer starken Community. Dies wird durch direkte Ansprache (“Ihr herzallerliebsten KnöpfchenDrückerInnen!”), Dankesbotschaften an Fans, Aufforderungen zum Teilen und Konsumieren (im Sinne von Musik hören/kaufen) sowie die Einbindung in Diskussionen (z.B. “Rock gegen Rechts”) erreicht.
4.  **Bandmitg

In [13]:
def generate_caption_with_style(captions, new_context, themes, model=MODEL):
    # Füge alle Captions zusammen (achte auf Token-Limit)
    joined_captions = "\n".join([f"- {caption}" for caption in captions[:10]])  # Nimm z.B. nur die ersten 20, um Token zu sparen

    # Erstelle Themen-Text
    themes_text = "\n".join([f"{i+1}. {theme}" for i, theme in enumerate(themes)])
    
    # Prompt mit Schreibstil + Themen + neuem Kontext
    prompt = (
        "Du bist Social Media Experte für eine Band. Analysiere zunächst den Schreibstil der folgenden Instagram-Captions "
        "und verwende diesen Stil für neue Vorschläge.\n\n"
        "Hier sind einige unserer erfolgreichsten Captions:\n"
        "---\n"
        f"{joined_captions}\n"
        "---\n\n"
        "Die folgenden Themen waren in unseren Posts besonders erfolgreich:\n"
        f"{themes_text}\n\n"
        "Wir möchten Fotos von folgendem Ereignis posten:\n"
        f"{new_context}\n\n"
        "💡 Schreibe 3 kreative Instagram-Captions, die:\n"
        "- Den Schreibstil aus den Beispielen oben übernehmen\n"
        "- Die erfolgreichen Themen aufgreifen\n"
        "- Emotional, mitreißend und passend für den Instagram-Auftritt einer Band sind\n"
        "- Emojis und bis zu 3 relevante Hashtags enthalten\n"
        "- Maximal 1–2 Sätze lang sind\n\n"
        "Gib die Captions als nummerierte Liste aus."
    )

    # Anfrage an LM Studio
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.95,  # Etwas kreativer
        max_tokens=500
    )
    
    return response.choices[0].message.content.strip()


# Beispiel-Daten
new_context = "Wir haben gerade ein Konzert auf dem Dalmstöckle Open Air gespielt und möchten dazu Bilder posten."
themes_for_creation = all_themes

print(themes_for_creation)

#captions = [post.caption for post in top_posts]  # Deine bestehenden Captions aus DB-Abfrage

# Aufruf
result = generate_caption_with_style(captions, new_context, themes_for_creation)
print("✨ Vorschläge für neue Captions:\n")
print(result)



['Okay, hier ist eine Analyse der Instagram-Captions von Dorfterror, die auf die Hauptthemen fokussiert ist und diese zusammenfasst:\n\n**Hauptthemen & Zusammenfassungen:**\n\n1.  **Musikalische Aktivität & Releases:** Dies ist das dominierendste Thema. Die Band stellt regelmäßig neue Musik (EPs, Musikvideos) vor und teilt Streaming-Links. Es geht um die aktive Verbreitung ihrer Musik.\n2.  **Live-Musik & Tournee:** Dorfterror legt großen Wert auf Live-Auftritte. Sie teilen regelmäßig Konzerttermine, Ankündigungen für Festivals und eigene Shows. Die Band betont das gemeinsame Musizieren und die Bühnenpräsenz.\n3.  **Community & Fan Engagement:** Ein zentraler Punkt ist der Aufbau einer starken Community. Dies wird durch direkte Ansprache (“Ihr herzallerliebsten KnöpfchenDrückerInnen!”), Dankesbotschaften an Fans, Aufforderungen zum Teilen und Konsumieren (im Sinne von Musik hören/kaufen) sowie die Einbindung in Diskussionen (z.B. “Rock gegen Rechts”) erreicht.\n4.  **Bandmitglieder & T

BadRequestError: Error code: 400 - {'error': 'Trying to keep the first 7495 tokens when context the overflows. However, the model is loaded with context length of only 4096 tokens, which is not enough. Try to load the model with a larger context length, or provide a shorter input'}